In [9]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table #as dt
from dash.dependencies import Input, Output, State

import base64 ##this is needed for the image
import plotly.graph_objects as go  #needed for my pie chart

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from aacanimal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "toulouse"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

print(df)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='row',
            style={'display' : 'flex'},
            children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                html.Button(id='submit-button-two', n_clicks=0, children='Mountain Rescue'),
                html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue'),
                html.Button(id='submit-button-four', n_clicks=0, children='Reset')
            ]),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        #filter_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])

#start Preferred Breeds
df = pd.DataFrame.from_records(shelter.read({
    "breed" :{"$in": ["Newfoundland", "Labrador Retriever Mix", "Chesapeake Bay Retriever"]},
    "sex_upon_outcome":{"$eq":"Intact Female"}, "age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}
}))

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-interactivity', "data")],
    [Input('submit-button-one', 'n_clicks'), Input('submit-button-two', 'n-_clicks'), 
     Input('submit-button-three', 'n_clicks'), Input('submit-button-four', 'n_clicks')]  
)
def on_click(bt1, bt2, bt3, bt4):
    #start case
    if (int(bt1)== 0 and int(bt2) ==0 and int(bt3) ==0 and int(bt4) ==0):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Water Rescue"})))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Mountain Rescue"})))
    elif (int(bt3) > int(bt4)):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Disaster Rescue"})))
    elif (int(bt4) > int(bt3)):
        df = pd.DataFrame(list(shelter.read({"Reset"})))
    return df.to_dict('records')

#[{
       # 'if': { 'column_id': i },
       # 'background_color': '#D2F3FF'
   # } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = ###
    #    )    
    #]
#start of pie chart
    @app.callback(
    Output("pie-chart", "figure"),
    [Input("names", "value"),
     Input("values", "value")])
    
    def generate_chart(names, values):
        fig = px.pie(df, values=values, names=names)
        return fig

labels = ['Border Collie','Boxer','Husky','St Bernard']
values = [4500, 2500, 1053, 500]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.show()
    
@app.callback(
        Output('datatable-interactivity-map', "children"),
        [Input('datatable-interactivity', "derived_viewport_data"),
         Input('datatable-interactivity', "derived_viewport_selected_rows")])

#Output('map-id', "children"),
#[Input('datatable-id', "derived_viewport_data")])

def update_map(viewData, rows):
        viewDF = pd.DataFrame.from_dict(viewData)
        dff = viewDF.loc[rows]
        
#@app.callback(
 #       Output('datatable-interactivity-container', "children"),
  #      [Input('datatable-interactivity', "derived_viewport_data")])
#def update_graphs(viewData):
    
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.

        dff = pd.DataFrame.from_dict(viewData)
    
    # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'30.75'],diff.loc[0,'-97.48']], children=[
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Reilly"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])]



app

  age_upon_outcome animal_id animal_type          breed            color  \
0           2 year   2020abc         Dog  Border Collie  Black and White   
1           2 year   2020abc         Dog  Border Collie  Black and White   
2           2 year   2020abc         Dog  Border Collie  Black and White   
3           2 year   2020abc         Dog  Border Collie  Black and White   
4           2 year   2020abc         Dog  Border Collie  Black and White   

  date_of_birth             datetime            monthyear    name  \
0    2018-09-04  2019-09-04 10:49:00  2019-09-04T10:49:00  Reilly   
1    2018-09-04  2019-09-04 10:49:00  2019-09-04T10:49:00  Reilly   
2    2018-09-04  2019-09-04 10:49:00  2019-09-04T10:49:00  Reilly   
3    2018-09-04  2019-09-04 10:49:00  2019-09-04T10:49:00  Reilly   
4    2018-09-04  2019-09-04 10:49:00  2019-09-04T10:49:00  Reilly   

  outcome_subtype outcome_type sex_upon_outcome  location_lat  location_long  \
0             PUP     Transfer             Male 